In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANG_SMITH")
os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY")

In [3]:
from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model="claude-3-sonnet-20240229")

In [4]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="Hi Bob, it's nice to meet you! I'm Claude, an AI assistant created by Anthropic. How are you doing today?", response_metadata={'id': 'msg_01D8HxUK2p3jLnSatKx4ZPHn', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 12, 'output_tokens': 32}}, id='run-0b29225f-64df-4c30-9350-ee586f566d2a-0', usage_metadata={'input_tokens': 12, 'output_tokens': 32, 'total_tokens': 44})

In [5]:
from langchain_core.messages import AIMessage

model.invoke([
    HumanMessage(content="Hi! I'm Bob"),
    AIMessage(content="Hello Bob!"),
    HumanMessage(content="What's my name?")
])

AIMessage(content='You said your name is Bob.', response_metadata={'id': 'msg_01SFNVf2JYmyWEsR6vQrNL5Z', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 26, 'output_tokens': 10}}, id='run-65c71f95-da09-4d4c-98f6-150d32797d59-0', usage_metadata={'input_tokens': 26, 'output_tokens': 10, 'total_tokens': 36})

In [6]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory
)
from langchain_core.runnables.history import RunnableWithMessageHistory

In [7]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [9]:
config = {"configurable": {"session_id":"abc2"}}

In [10]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config
)

response.content

"Hello Bob, nice to meet you! I'm Claude, an AI assistant created by Anthropic. How are you doing today?"

In [12]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name")],
    config=config
)

response.content

'You introduced yourself as Bob, so based on that, your name appears to be Bob. However, since I\'m an AI assistant without access to additional information about you, I can\'t confirm or provide any other names you might have beyond "Bob."'

In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability."  
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [16]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})
response.content

"Hi Bob, nice to meet you! I'm Claude, an AI assistant created by Anthropic. How are you doing today?"

In [18]:
response = with_message_history.invoke(
    [HumanMessage(content="hi! I'm bob")],
    config=config,
)

response.content

'Hello again Bob. I understand that you previously introduced yourself as Bob, so I have that recorded as your name. Is there anything else I can help you with today?'

In [19]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [21]:
response = chain.invoke(
    {"messages":[HumanMessage(content="hi! I'm bob")], "language":"Korean"}
)

response.content

'안녕하세요 밥씨, 저는 Claude라는 인공지능 대화상대입니다. 한국어로 대화할 수 있어서 기쁩니다. 어떤 도움이 필요하신가요?'

In [22]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [23]:
config = {"configurable": {"session_id":"abc11"}}

In [24]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language":"Korean"},
    config=config,
)
response.content

'안녕하세요 토드씨. 저는 한국어로 대화를 잘 할 수 있는 도움이 되는 AI 어시스턴트입니다. 편안하게 어떤 질문이든 물어봐 주시기 바랍니다. 최선을 다해 답변해 드리겠습니다.'

In [25]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [28]:
from operator import itemgetter

In [ ]:
itemgetter()()

In [30]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")

|Okay, here's| another| joke| for you,| Todd:

Why| di|d the tomato turn| red? Because it| saw| the salad |dressing!||

1. 프롬프트 템플릿 작성 ChatPromptTemplate
2. LLM 모델 등록 ChatAnthropic
3. 체인 생성 및 stores dict 생성 BaseChatMessageHistory
4. 메시지 히스토리 런너블 객체 생성 RunnableWithMessageHistory
5. 연결하기